In [195]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import os
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import time

In [2]:
court_directory = ("/Users/shalakashinde/Columbia/master_project/court_page_htmls")
court_files = os.listdir(court_directory)
len(court_files)

179

### TO DO LIST FOR JULY 26

In [39]:
#Run the original scraper script to get the name of the district
#Then get the CNR directly without performing the steps between district and CNR extraction
#In the same script split the CNR into alpha and digit
#Replace UP in alpha with ""
#Save this new string as "abbr"
#Make new file with district and abbr
#Save it as csv to join with final df

### TO DO LIST FOR JULY 27

In [40]:
#Save error mesages in a separate directory
#Look for patterns of which ones have erroerd out
#Know how many errored out

In [3]:
def case_htmls_to_df(files):
    #Read into dict
    #Make it a list of everything in it
    directory = ("/Users/shalakashinde/Columbia/master_project/each_case_htmls")
    files = os.listdir(directory)

    #iterate through the list and read each html
    #start a for loop
    list_for_sanity = []
    for each_case in tqdm(files):
        case_dates_dict = {}
        # Opening the html file
        HTMLFile = open(f"each_case_htmls/{each_case}", 'r')

        # Reading the file
        index = HTMLFile.read()

        # Creating a BeautifulSoup object and specifying the parser (using laxml instead of html.parser because files are local)
        S = BeautifulSoup(index, 'lxml')

        #Extracting CNR from each file
        CNR_list = str(each_case).split(".")
        CNR = CNR_list[0]

        #Extracting case type
        check = S.find('td', class_='text-uppercase')
        if check:
            case_type = check.text.strip()

        else:
            case_type = "NA"
            
        
        #Extracting court name


        #Extracting court number
        court_number_check = S.find('td', text='Court Number and Judge')
        if court_number_check:
            court_number = S.find('td', text='Court Number and Judge').find_next('td').text.strip()
        else:
            court_number = "NA"   
            

        #Extracting court type
        try:
            court_type = S.find('h2').text.strip()
        except:
            court_type = "NA"

        #Extarcting petitioner and respondent
        petitioner_advocate_check = S.find(class_='Petitioner_Advocate_table')
        respondent_advocate_ckeck = S.find(class_='Respondent_Advocate_table')
        if petitioner_advocate_check:
            petitioner_advocate = S.find(class_='Petitioner_Advocate_table').text.strip()
        else:
            petitioner_advocate = "NA"
        if respondent_advocate_ckeck:
            respondent_advocate = S.find(class_='Respondent_Advocate_table').text.strip()
        else:
            respondent_advocate = "NA"

        #Extracting Registration Date
        registration_date_check = S.find('td', text='Registration Date:')
        if registration_date_check:
            registration_date = S.find('td', text='Registration Date:').find_next('td').text.strip()
        else:
            registration_date = "NA"


        # Extracting First Hearing Date
        first_hearing_date_check = S.find('td', text='First Hearing Date')
        if first_hearing_date_check:
            first_hearing_date = S.find('td', text='First Hearing Date').find_next('td').text.strip()
        else:
            first_hearing_date = "NA"

        # Extracting Next Hearing Date
        next_hearing_date_check = S.find('td', text='Next Hearing Date')
        if next_hearing_date_check:
            next_hearing_date = next_hearing_date_check.find_next('td').text.strip()
        else:
            next_hearing_date = "NA"


        # Extracting Case Stage
        case_stage_check = S.find('td', text='Case Stage')
        if case_stage_check:
            case_stage = case_stage_check.find_next('td').text.strip()
        else:
            case_stage = "NA"

        case_dates_dict["CNR"] = CNR
        case_dates_dict["case_type"] = case_type
        case_dates_dict["court_type"] = court_type
        case_dates_dict["Court Number"] = court_number
        case_dates_dict["petitioner_advocate"] = petitioner_advocate
        case_dates_dict['respondent_advocate'] = respondent_advocate
        case_dates_dict["Registration Date"] = registration_date
        case_dates_dict["First Hearing Date"] = first_hearing_date
        case_dates_dict["Next Hearing Date"] = next_hearing_date
        case_dates_dict["Case Stage"] = case_stage    

        list_for_sanity.append(case_dates_dict)
    return list_for_sanity

In [4]:
directory = ("/Users/shalakashinde/Columbia/master_project/each_case_htmls")
files = os.listdir(directory)
list_for_df = case_htmls_to_df(files)

  0%|          | 0/38413 [00:00<?, ?it/s]

In [43]:
len(list_for_df)

38413

In [193]:
district_names = pd.read_csv('district_abbr.csv')
district_names['abbr'] = district_names['abbr'].str.replace(" ", "")

In [5]:
df = pd.DataFrame(list_for_df)
df.shape

(38413, 10)

In [6]:
df.head(1)

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage
0,UPBR010007862023,Criminal Misc. Cases,District and Session Judge,NA,1) STATE OF U.P Advocate- Mr. Dubey(ADGC12),1) Shubham Kumavat,20-01-2023,27th January 2023,27th January 2023 Date Not Updated,Objection/Disposal


In [46]:
session = df[df['case_type'].str.contains(r'Session', case=False)].reset_index(drop=True)
session.head()

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage
0,UPLL010001112020,Sst_ - Special Session Trial,District and Session Judge,3-Spl. Judge(U.P. Dacoity.A.A.),"1) STATE OF U.P. Advocate- DGC Crl,",1) Sri Rahul Rajak And. Others,02-12-2014,,21st August 2023,Evidence
1,UPLK010141542021,Sessions Case,District and Session Judge,1-District & Session Judge,1) State of UP,1) ISHRAT ALI AND OHTERS,11-11-2021,26th November 2021,18th August 2023,Prosecution Evidence
2,UPBR010016112021,Sessions Case,District and Session Judge,16-XII th ADJ,1) STATE OF U.P Advocate- DGC(Crl),1) Pramod Kumar,23-02-2021,24th February 2021,21st August 2023,Evidence
3,UPRB010009722014,Session Trial,District and Session Judge,5-Spl. Judge(E.C. Act),1) State Government Advocate- ADGC,1) ARUN SHARMA Advocate - VIMAL KUMAR SRIVAS...,05-06-2014,01st July 2014,16th August 2023,Evidence
4,UPMT010173122019,Session Trial,"District and Session Judge, Mathura",4-Addl. District Judge Court No. 3,1) State Government Advocate- a,1) Janak alias Bintu Devraj,02-12-2019,04th December 2019,31st July 2023,Evidence


In [117]:
# CNR = ["UPJP040116682023", "UPBH040174152018", "UPGB010126842017", "UPGB040098632019", "UPFZ040011152020"]

def splitting_CNR_for_alpha(input_string):
    alphabets = ""

    for char in input_string:
        if char.isalpha():
            alphabets += char
            district_code = alphabets[2:]
        
    return district_code

In [101]:
# CNR = ["UPJP040116682023", "UPBH040174152018", "UPGB010126842017", "UPGB040098632019", "UPFZ040011152020"]
def splitting_CNR_for_year(input_string):
    digits = ""

    for char in input_string:
        if char.isdigit():
            digits += char
            year = digits[8:]
    
    return year

In [47]:
session.shape

(20736, 10)

In [52]:
#These will be useless because year of the first hearing date is missing a digit
#Dropping these from the data
session[session['First Hearing Date'].str.contains(r"201\b")]
session = session.drop(labels=[4821, 5213, 7736, 9854, 14516]).reset_index(drop=True)
session.head()

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage
0,UPLL010001112020,Sst_ - Special Session Trial,District and Session Judge,3-Spl. Judge(U.P. Dacoity.A.A.),"1) STATE OF U.P. Advocate- DGC Crl,",1) Sri Rahul Rajak And. Others,02-12-2014,,21st August 2023,Evidence
1,UPLK010141542021,Sessions Case,District and Session Judge,1-District & Session Judge,1) State of UP,1) ISHRAT ALI AND OHTERS,11-11-2021,26th November 2021,18th August 2023,Prosecution Evidence
2,UPBR010016112021,Sessions Case,District and Session Judge,16-XII th ADJ,1) STATE OF U.P Advocate- DGC(Crl),1) Pramod Kumar,23-02-2021,24th February 2021,21st August 2023,Evidence
3,UPRB010009722014,Session Trial,District and Session Judge,5-Spl. Judge(E.C. Act),1) State Government Advocate- ADGC,1) ARUN SHARMA Advocate - VIMAL KUMAR SRIVAS...,05-06-2014,01st July 2014,16th August 2023,Evidence
4,UPMT010173122019,Session Trial,"District and Session Judge, Mathura",4-Addl. District Judge Court No. 3,1) State Government Advocate- a,1) Janak alias Bintu Devraj,02-12-2019,04th December 2019,31st July 2023,Evidence


In [57]:
#Along with the 5 above (for not having correct registration year), these 31 will also be have 
session[session['Next Hearing Date'] == "NA"].shape

session['next_hearing_dt'] = pd.to_datetime(session['Next Hearing Date'], errors='coerce')

(31, 11)

In [58]:
session['first_hearing_dt'] = pd.to_datetime(session['First Hearing Date'], errors="ignore")
session['Next Hearing Date'] = session['Next Hearing Date'].str.replace("Date Not Updated", "")
session['registration_dt'] = pd.to_datetime(session['Registration Date'], format='%d-%m-%Y')

In [60]:
session['registration_year'] = session['registration_dt'].dt.year
session['first_hearing_year'] = session['first_hearing_dt'].dt.year

In [69]:
session['reg_to_first_years'] =  session['first_hearing_year'] - session['registration_year']

## Checking age of the case using registration year

In [146]:
from datetime import datetime

In [149]:
session['age_in_2023'] = datetime.now().year - session['registration_year']

In [155]:
session.groupby('age_in_2023')['CNR'].count()

age_in_2023
0     1580
1     3011
2     3093
3     1943
4     2260
5     2222
6     1712
7      981
8      611
9      809
10     653
11     435
12     320
13     242
14     163
15     143
16     142
17      90
18      74
19      71
20      45
21      21
22      33
23      14
24      19
25      11
26       7
28       4
29       2
30       2
31       2
32       2
33       2
34       3
35       1
38       1
40       2
Name: CNR, dtype: int64

In [160]:
decade_older = session[session['age_in_2023'] >= 10].reset_index(drop=True)
decade_older.head(2)

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage,first_hearing_dt,registration_dt,registration_year,first_hearing_year,reg_to_first_years,age_in_2023
0,UPSH010000242009,Session Trial,District and Session Judge,"14-Additional District and Sessions Judge,Cour...",1) State Government Advocate- DGC,1) BHAIYA LAL Advocate - R.Giri2) HANS RAM ...,28-05-2009,03rd October 2012,27th July 2023,Evidence,2012-10-03,2009-05-28,2009,2012.0,3.0,14
1,UPBD010011842013,Session Trial,District and Session Judge,2-ADJ I,1) State Govt. of U.P.,1) Lavkush Singh and others Advocate - SHANK...,24-12-2013,29th July 2013,07th August 2023,Rest Evidence,2013-07-29,2013-12-24,2013,2013.0,0.0,10


In [163]:
decade_older['district_code'] = decade_older['CNR'].apply(splitting_CNR_for_alpha)

In [165]:
decade_older['district_code'].nunique()

58

## Separating session cases that have taken more than 1 year after registration to be heard

In [138]:
reg_to_first = session[session['reg_to_first_years'] != 0].reset_index(drop=True)
reg_to_first.sort_values(by='reg_to_first_years', ascending=False).head(15)

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage,first_hearing_dt,registration_dt,registration_year,first_hearing_year,reg_to_first_years
1954,UPVR010000151983,Session Trial -Old - Session Trial,District and Session Judge,10-Vth Additional District and Sessions Judge,1) State Of U.P.,1) Ashok,16-08-1983,15th March 2023,07th August 2023,Appearance,2023-03-15,1983-08-16,1983,2023.0,40.0
1059,UPGH010000031983,Session Trial,District and Session Judge,9-Special Judge (EC Act),1) U.P.State Advocate- A.P.O.,1) Sudama,05-08-1983,23rd October 2020,09th August 2023,Evidence,2020-10-23,1983-08-05,1983,2020.0,37.0
3116,UPGH010000251989,Session Trial,District and Session Judge,24-ADJ I,1) U.P.State Advocate- A.P.O.,1) Gulab Chandra Ram,17-07-1989,18th May 2023,28th August 2023,Evidence,2023-05-18,1989-07-17,1989,2023.0,34.0
1666,UPHR010000051985,Session Trial,District and Session Judge,2-Addl. District And Sessions Judge Court No.1,1) State Government,1) Babban,20-05-1985,31st August 2018,27th July 2023,F. O.,2018-08-31,1985-05-20,1985,2018.0,33.0
630,UPRB010000151990,Session Trial,District and Session Judge,16-Addl District and Session Judge (POCSO Act)-II,1) State Government Advocate- ATUL KUMAR SINGH,1) mo. sabir,02-11-1990,05th July 2018,29th August 2023,Evidence,2018-07-05,1990-11-02,1990,2018.0,28.0
734,UPRN010000071990,Session Trial,District and Session Judge,9-IVth Additional District and Sessions Judge,1) State,1) SHYAM SUNDER AND OTHER,11-06-1990,30th November 2017,05th August 2023,Appearance,2017-11-30,1990-06-11,1990,2017.0,27.0
2295,UPRN010000351993,Session Trial,District and Session Judge,9-IVth Additional District and Sessions Judge,1) State,1) HARI KISHAN,06-05-1993,30th November 2017,05th August 2023,Summoning,2017-11-30,1993-05-06,1993,2017.0,24.0
123,UPBR010000121992,Sessions Trial,District and Session Judge,1-District Judge,1) State of U.P Advocate- Akhilesh,1) Girdhari Lal Sahu Alias Pappu Advocate - ...,05-02-1992,03rd May 2016,31st July 2023,Appearance,2016-05-03,1992-02-05,1992,2016.0,24.0
1765,UPAD010003771995,SESSIONS TRIAL,District and Session Judge,3-Special Judge EC Act,1) State of U.P. Advocate- ADGC(CRI),1) IMRAN AND OTHERS,04-12-1995,02nd August 2019,31st July 2023,Appearance,2019-08-02,1995-12-04,1995,2019.0,24.0
902,UPMZ010000041989,Session Trial - Session Case,District and Session Judge,44-Addl. Distt. and Sess. Judge F.T.C No. 3,1) State Of U.P. Advocate- A.D.G.C. (Criminal),1) Devender Advocate - Anil Kumar,10-07-1989,28th July 2012,03rd August 2023,F. O.,2012-07-28,1989-07-10,1989,2012.0,23.0


In [139]:
reg_to_first.shape

(3197, 15)

In [134]:
reg_to_first.head(1)

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage,first_hearing_dt,registration_dt,registration_year,first_hearing_year,reg_to_first_years,CNR_alpha,CNR_dig
0,UPLL010001112020,Sst_ - Special Session Trial,District and Session Judge,3-Spl. Judge(U.P. Dacoity.A.A.),"1) STATE OF U.P. Advocate- DGC Crl,",1) Sri Rahul Rajak And. Others,02-12-2014,,21st August 2023,Evidence,NaT,2014-12-02,2014,NaN,NaN,LL,2020


In [130]:
reg_to_first[~reg_to_first['Case Stage'].str.contains(r"evidence", case=False)].head(1)

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage,first_hearing_dt,registration_dt,registration_year,first_hearing_year,reg_to_first_years,CNR_alpha,CNR_dig
10,UPMA010000012007,Session Trial,District and Session Judge,14-Addl. District and Sessions Judge/ F.T.C.II,1) State Of Uttar Pradesh Advocate- A.D.G.C.Cri,1) Sarwan Advocate - Vishnu Datt Rai2) lagg...,22-05-2014,12th May 2011,31st July 2023,Appearance,2011-05-12,2014-05-22,2014,2011.0,-3.0,MA,2007


In [221]:
reg_to_first = reg_to_first[reg_to_first['reg_to_first_years'] > 0]
reg_to_first.head(1)

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage,first_hearing_dt,registration_dt,registration_year,first_hearing_year,reg_to_first_years
1,UPHM010016602014,Session Trial,District and Session Judge,18-A.D.J./ Spl. Judge POCSO Act.,1) State Of U.P . Advocate- adgc,1) IMAMI ALIAS IMAM KHA AND 1 ANRS. Advocate...,21-08-2014,10th July 2017,02nd August 2023,Defence Evidence,2017-07-10,2014-08-21,2014,2017.0,3.0


In [197]:
district_names = pd.read_csv('district_abbr.csv')
district_names.columns = district_names.columns.str.replace(" ", "")
district_names

,district,abbr
0,Agra,AG
1,Aligarh,AL
2,Ambedkar Nagar,AN
3,Amroha,JB
4,Auraiya,AU
...,...,...
69,Sitapur,SI
70,Sonbhadra,SB
71,Sultanpur,ST
72,Unnao,UN


In [265]:
district_names[district_names['abbr'] == "HR"]

,district,abbr
34,Hardoi,HR


In [254]:
district_names.abbr =district_names.abbr.str.strip()

In [225]:
reg_to_first['abbr'] = reg_to_first['CNR'].apply(splitting_CNR_for_alpha)
reg_to_first['CNR_dig'] = reg_to_first['CNR'].apply(splitting_CNR_for_year)

In [255]:
merged_df = pd.merge(reg_to_first, district_names, on='abbr', how='left')
merged_df

,CNR,case_type,court_type,Court Number,petitioner_advocate,respondent_advocate,Registration Date,First Hearing Date,Next Hearing Date,Case Stage,first_hearing_dt,registration_dt,registration_year,first_hearing_year,reg_to_first_years,CNR_dig,abbr,district
0,UPHM010016602014,Session Trial,District and Session Judge,18-A.D.J./ Spl. Judge POCSO Act.,1) State Of U.P . Advocate- adgc,1) IMAMI ALIAS IMAM KHA AND 1 ANRS. Advocate...,21-08-2014,10th July 2017,02nd August 2023,Defence Evidence,2017-07-10,2014-08-21,2014,2017.0,3.0,2014,HM,Hamirpur
1,UPSH010000242009,Session Trial,District and Session Judge,"14-Additional District and Sessions Judge,Cour...",1) State Government Advocate- DGC,1) BHAIYA LAL Advocate - R.Giri2) HANS RAM ...,28-05-2009,03rd October 2012,27th July 2023,Evidence,2012-10-03,2009-05-28,2009,2012.0,3.0,2009,SH,Shahjahanpur
2,UPBU130001072017,SESSIONS TRIAL,"Addl. District Judge, Khurja",1-Addl. District and Sessions Judge,1) STATE OF UTTAR PRADESH Advocate- A.D.G.C,1) JEETU URF JEETPAL,25-05-2017,12th March 2018,11th August 2023,Evidence,2018-03-12,2017-05-25,2017,2018.0,1.0,2017,BU,Bulandshahr
3,UPKS010041942015,S T - Session Trial,District and Session Judge,19-Additional District and Session Judge RNo. 7,1) State Government,1) ram chandra pal2) Gopi nath 3) Namo Na...,26-06-2015,30th June 2017,27th July 2023,Evidence,2017-06-30,2015-06-26,2015,2017.0,2.0,2015,KS,Kaushambi
4,UPKS010017732013,S T - Session Trial,District and Session Judge,21-Additional District and Session Judge RNo. 9,1) State Government,1) gulab pasi2) guddu pasi,18-09-2013,02nd July 2015,10th August 2023,Rest Evidence,2015-07-02,2013-09-18,2013,2015.0,2.0,2013,KS,Kaushambi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,UPCD010001022010,SESSIONS TRIAL,District & session Judge,4-Additional District & Session Judge RNo. 2,1) State of U.P. Advocate- D.G.C.,1) Keshari Singh and others2) Ajit singh 3...,17-05-2010,01st August 2018,27th July 2023,Statement U/S 313,2018-08-01,2010-05-17,2010,2018.0,8.0,2010,CD,Chandauli
2438,UPPG010000191998,Session Trial,District and Session Judge,"42-Special Judge, SC/ST",1) State Government Advocate- Special public...,1) hori lal Advocate - Badri Prasad Chaurasi...,04-02-1998,06th June 2012,28th July 2023,Evidence,2012-06-06,1998-02-04,1998,2012.0,14.0,1998,PG,Pratapgarh
2439,UPBH010016872011,Session Trial,District and Session Judge,31-Additional District and Sessions Judge/ Spe...,1) STATE OF UP Advocate- D.G.C.CRIMINAL,1) JAKIR,11-04-2017,12th March 2018,01st August 2023,Evidence,2018-03-12,2017-04-11,2017,2018.0,1.0,2011,BH,Bahraich
2440,UPMO010006092008,Session Trial(Old),District and Session Judge,"13-Addl. District and Sessions Judge, Court No...",1) U P State Advocate- ADGC,1) Shiv Swaroop Advocate - H N Kapur,02-02-2009,30th January 2012,09th August 2023,Prosecution Evidence,2012-01-30,2009-02-02,2009,2012.0,3.0,2008,MO,Moradabad


# How Many of the Stuck Court Matters Are Trials?
* The answer is over 20,100
* This can be guaged with session.shape

# Which are Some of the Oldest Cases?
* The oldest case is in Varanasi. It has been ongoing for 40 years
* There are 3 cases that are over 30 years old but less than 40. Two of them are in Ghazipur and one in Hardoi
* There are hundreds of cases that have been in the system for anywhere between 10 years to 30 years

# Where Are Most Stuck Trials Concentrated?
* The answer is: Shahjahanpur, Lakhimpur Kheri, Muzaffarnagar, Budaun, Etah, Prayagraj, Kushinagar.
* All these districts have over 100 trials that have 

In [262]:
merged_df.groupby('district').CNR.count().reset_index().sort_values(by='CNR', ascending=False).head(10)

,district,CNR
54,Shahjahanpur,155
36,Lakhimpur Kheri,136
45,Muzaffarnagar,135
13,Budaun,133
18,Etah,113
48,Prayagraj,108
35,Kushinagar,104
20,Farrukhabad,87
44,Moradabad,85
15,Chandauli,72


# What is the Judicial Vacancy of These Districts?
* This answer needs more data from the judicial dashboard
* For this project, only state-aggregate judicial vacancy numbers are available